In [9]:
import openai
import cohere
import os
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.http.models import models
from dotenv import load_dotenv

load_dotenv()

collection_name = "hackathon_collection"

# Title: The Will to Believe and Other Essays in Popular Philosophy
# Author: William James

def search_by_filter(key, value, question):
    # perform author payload filter + information vector search
    cohere_client = cohere.Client(os.environ.get('COHERE_API_KEY'))
    model = 'multilingual-22-12'
    response = cohere_client.embed(
        texts=[question],
        model=model,
    )
    embedding = [float(e) for e in response.embeddings[0]]
    db_client = QdrantClient(
        api_key=os.environ.get('QDRANT_API_KEY'),
        host=os.environ.get('QDRANT_HOST')
    )
    response = db_client.search(
        collection_name=collection_name,
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key=key,
                        match=models.MatchValue(
                            value=value
                        ) 
                    )
                ]
            ),
        query_vector=embedding,
        limit=5
    )

    return response[0].payload.get(key)

def search_author(input):
    author_info, question_info = input.split('AUTHOR:', 1)[1].split('INFORMATIONS:', 1)
    author = author_info.strip().lower()
    question = question_info.strip().lower()
    return search_by_filter(key='author', value=author, question=question)


def search_title(input):
    # perform title payload filter + information vector search
    title_info, question_info = input.split('TITLE:', 1)[1].split('INFORMATIONS:', 1)
    title = title_info.strip().lower()
    question = question_info.strip().lower()
    return search_by_filter(key='title', value=title, question=question)

def search(question):
    cohere_client = cohere.Client(os.environ.get('COHERE_API_KEY'))
    model = 'multilingual-22-12'
    response = cohere_client.embed(
        texts=[question],
        model=model,
    )

    embedding = [float(e) for e in response.embeddings[0]]

    db_client = QdrantClient(
        api_key=os.environ.get('QDRANT_API_KEY'),
        host=os.environ.get('QDRANT_HOST')
    )
    qdrant_answer = db_client.search(
        collection_name="hackaton_collection",
        query_vector=embedding,
        limit=5,
    )

    prompt = f"""
        Given the texts below, answer the following question:
        Question: {question}

        Texts:
        """
    for response in qdrant_answer:
        prompt += '{}\n'.format(response.payload.get('text'))
    
    messages = [{"role": "user", "content": prompt}]
    
    openai_model = 'gpt-3.5-turbo'
    response = openai.ChatCompletion.create(
        model=openai_model,
        messages=messages,
        temperature=0.1,
        max_tokens=1000,
        # frequency_penalty=0.0,
        # presence_penalty=0.0,
        # stop=["\n"]
    )
    return response.choices[0].message.content


tools = [
    Tool(
        name="Search",
        func=lambda question: search(question),
        description="useful for when you need to answer questions and you do not have an author name or you do not know the title"
        # description="use when searching for information filtering by a specific author.",
        # description="use when you want to discover who is the author, asking a question with informations you have",
    ),
    Tool(
        name="Author Search",
        func=lambda input: search_author(input),
        description="use this more than normal search when the author name is mentioned in the question. Create an input with the name of the author and more information about the question. Input must be like: 'AUTHOR: name of the author INFORMATIONS: other informations you have'"
        # description="use when you know the author's name and want to filter results based on their name and other informations that you have. create input like 'author: information:'"
        # description="use when searching for information filtering by a specific author.",
        # description="use when you want to discover who is the author, asking a question with informations you have",
    ),
    Tool(
        name="Title Search",
        func=lambda input: search_title(input),
        description="use this when the you already know the title of the book. Create an input with the name of the author and more information about the question. Input must be like: 'TITLE: title INFORMATIONS: other informations you have'"
        # description="use when searching for information filtering by a specific title.",
        # description="use when you want to discover which is the title, asking a quesiton with informations you have",
    )
]

agent = initialize_agent(
    tools=tools, 
    llm=OpenAI(temperature=0), 
    agent="zero-shot-react-description", 
    verbose=True,
)



In [10]:
# question = 'quem é o autor que escreve sobre suas memórias póstumas?'
question = 'em qual trecho o machado de assis comenta sobre filhos?'
# question = 'qual é o titulo do livro em o autor machado de assis comenta sobre filhos?'
# question = 'Com efeito, um dia de manhã, estando a passear na chácara, pendurou-se-me uma idéia no trapézio que eu tinha no cérebro.'

agent.run(input=question)



> Entering new AgentExecutor chain...
 I need to find a book by Machado de Assis that talks about children.
Action: Author Search
Action Input: AUTHOR: Machado de Assis INFORMATIONS: children
Observation: machado de assis
Thought: I need to find a specific book by Machado de Assis that talks about children.
Action: Title Search
Action Input: TITLE: Memórias Póstumas de Brás Cubas INFORMATIONS: children
Observation: memórias póstumas de brás cubas
Thought: I need to find the specific passage in the book that talks about children.
Action: Search
Action Input: Memórias Póstumas de Brás Cubas INFORMATIONS: children
Observation: There is no information about children in the given texts.
Thought: I need to look for other books by Machado de Assis that talk about children.
Action: Author Search
Action Input: AUTHOR: Machado de Assis INFORMATIONS: children
Observation: machado de assis
Thought: I need to find a specific book by Machado de Assis that talks about children.
Action: Title Search

IndexError: list index out of range